In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from mip.model import *

In [ ]:
# importing the data to take a look at what we have

df = pd.read_csv('../data/family_data.csv',index_col=0)

df.head()

In [ ]:
# load the penalties for days with high traffic
day_penalty = pd.read_csv('day_penalty.txt',sep='\t',index_col=0)['penalty']
day_penalty.head()

In [ ]:
day_penalty[5]

In [ ]:
# for testing
num_days=100
num_families = 5000

In [ ]:
df2 = df[:num_families].copy()

In [ ]:
#people scaling 
people_scale = df2.n_people.sum()/df.n_people.sum()

max_people = np.around(1.5*df2.n_people.sum()/num_days)
min_people = np.around(df2.n_people.sum()/2/num_days)

if num_days==100 and num_families==5000:
    max_people = 300
    min_people = 125

# day scaling
day_scale = num_days/100

In [ ]:
print(max_people)
print(min_people)
print(day_scale)
print(people_scale)

In [ ]:
if num_days!=100:
    for c in df2.columns.tolist()[:10]:
        df2[c] = np.random.randint(1,num_days+1,num_families)

In [ ]:
df2.head()

In [ ]:
# I'm going to add a column which will represent the choice falling outside of any of the given choices. 
# This will be choice_10, and it will be 101,,,

In [ ]:
df2.describe()

In [ ]:
df2['choice_0'].hist(bins=[b for b in range(1,num_days+2,1)])

In [ ]:
lower_limit = min_people
upper_limit = max_people

# creating the decision variables
choice = ['choice_0','choice_1', 'choice_2', 'choice_3', 'choice_4', 'choice_5', 
          'choice_6', 'choice_7', 'choice_8', 'choice_9','choice_10']
day = [i for i in range(1,num_days+1)]
fam_id = df2.index.tolist()
n_people = df['n_people'].to_dict()
npd = [n for n in range(min_people,max_people+1,1)]

In [ ]:
choice_matrix = {}

for f in fam_id:
    
    if f%100==0:
        print('{}/{}'.format(f,len(df2)))
    
    choice_matrix[f] = {}
    
    for d in day:
        
        choice_to_check = df2.loc[f,df2.columns.tolist()[:10]].values
        
        if d in choice_to_check:
            c_loc = np.where(choice_to_check==d)[0][0]
        else:
            c_loc = 10
            
            
        choice_matrix[f][d] = {c:(0 if i!=c_loc else 1) for i,c in enumerate(choice)} 

In [ ]:
# gift card contribution by choice
choice_gc = {}
choice_gc['choice_0'] = 0
choice_gc['choice_1'] = 50
choice_gc['choice_2'] = 50
choice_gc['choice_3'] = 100
choice_gc['choice_4'] = 200
choice_gc['choice_5'] = 200
choice_gc['choice_6'] = 300
choice_gc['choice_7'] = 300
choice_gc['choice_8'] = 400
choice_gc['choice_9'] = 500
choice_gc['choice_10'] = 500

# per member monetary contribution
choice_pm = {}
choice_pm['choice_0'] = 0
choice_pm['choice_1'] = 0
choice_pm['choice_2'] = 9
choice_pm['choice_3'] = 9
choice_pm['choice_4'] = 9
choice_pm['choice_5'] = 18
choice_pm['choice_6'] = 18
choice_pm['choice_7'] = 36
choice_pm['choice_8'] = 36
choice_pm['choice_9'] = 36+199
choice_pm['choice_10'] = 36+398


Create a lookup table for the accounting penalty

In [ ]:
def accounting_penalty_actual(Nd,Nd1):
    diff = np.abs(Nd-Nd1)
    return 300/max_people*(Nd-min_people)/400 * Nd**(0.5+diff/50)

In [ ]:
acc_table = {}
for Nd in npd:
    for Nd1 in npd:
        acc_table[(Nd,Nd1)] = accounting_penalty_actual(Nd,Nd1)

In [ ]:
acc_table[(300,125)]

In [ ]:
plt.plot(npd,[acc_table[(300,i)] for i in npd])

The decision variable needs to be a boolean for each choice for each family. We will create a 

In [ ]:
# set the parameters

max_diff = 35 # this is the maximum difference between two days in total number of people
w1 = 0 # this is the weight applied to the simple difference ppd(d)-ppd(d+1)
w2 = 0 # this is the weight applied to the simple linear shopping penalty
w3 = 10 # this is the multiplier for the day penalty for high traffic days

In [ ]:
# The prob variable is created to contain the problem data        
m = Model()

In [ ]:
# The decision variables are actually the family and the day they are assigned
x = [ [m.add_var(name='fam_{},day_{}'.format(f,d),var_type=BINARY) for d in day] for f in fam_id ]

In [ ]:
y = [ [ [m.add_var(name='d_{}_nd_{}_nd1_{}'.format(d,nd,nd1),var_type=BINARY)#INTEGER, lb=min_people, ub=max_people)
         for nd1 in npd]
       for nd in npd]
     for d in day]

In [ ]:
def ppd_fast(di):
    if di==num_days:
        di = num_days-1
    return xsum(x[fi][di]*n_people[f] for fi,f in enumerate(fam_id))

def ppd(d):
    if d>num_days:
        d=num_days
    di = day.index(d)
    return xsum(x[fi][di]*n_people[f] for fi,f in enumerate(fam_id))

In [ ]:
m.objective = minimize(xsum(x[fi][di]*choice_matrix[f][d][c]*(choice_gc[c] + n_people[f]*choice_pm[c])
                       for c in choice for di,d in enumerate(day) for fi,f in enumerate(fam_id))
                      + xsum(y[di][ndi][nd1i]*acc_table[(nd,nd1)] 
                             for nd1i,nd1 in enumerate(npd) 
                             for ndi,nd in enumerate(npd) 
                             for di,d in enumerate(day)))

#m.objective = minimize(xsum(x[fi][di]*choice_matrix[f][d][c]*(choice_gc[c] + n_people[f]*choice_pm[c])
#                            + ppd_fast(di)*(w1+w2) - w1*ppd_fast(di+1) - w2*125
#         for c in choice for di,d in enumerate(day) for fi,f in enumerate(fam_id))) 

#m.objective = minimize(xsum(x[fi][di]*choice_matrix[f][d][c]*(choice_gc[c] + n_people[f]*choice_pm[c]+w3*day_penalty[d])
#                       for c in choice for di,d in enumerate(day) for fi,f in enumerate(fam_id)))
                       
                      #+ xsum(ppd_fast(di)*(w1+w2) - w1*ppd_fast(di+1) - w2*125 for di,d in enumerate(day))) 

In [ ]:
# adding in the constraints

# The first set of constraints ensures each family only has a single day selected
for fi,f in enumerate(fam_id):
        m += xsum(x[fi][di] for di,d in enumerate(day)) == 1

In [ ]:
# the second set of constraints guarantee that the total number of visitors is between 125 and 300 for
# for every single day leading up to christmas

for di,d in enumerate(day):
    m += ppd(d) >= lower_limit, ''
    m += ppd(d) <= upper_limit, ''

In [ ]:

#my[d-1][ndi][nd1i] for nd1i,nd1 in enumerate(npd) for ndi,nd in enumerate(npd)

for di,d in day[:]:
    
    # each day should only have 1 entry
    m += xsum(y[di][ndi][nd1i] for nd1i,nd1 in enumerate(npd) for ndi,nd in enumerate(npd)) == 1
    
    # the number of people on day d needs to match
    m += ppd(d) == xsum(y[di][ndi][nd1i]*nd for nd1i,nd1 in enumerate(npd) for ndi,nd in enumerate(npd))
    
for di,d in day[:-1]
    # the number of people on the next day in the sum needs to match the next day
    m += ppd(d+1) == xsum(y[di][ndi][nd1i]*nd1 for nd1i,nd1 in enumerate(npd) for ndi,nd in enumerate(npd))
    
# the last day needs to have the next day set to the last day number of people
m += ppd(d[-1]) == xsum(y[-1][ndi][nd1i]*nd1 for nd1i,nd1 in enumerate(npd) for ndi,nd in enumerate(npd))

In [ ]:
# adding this third constraint to prevent the difference between each day from climbing too high.

#for di,d in enumerate(day[0:len(day)-1]):
#    m += ppd(d)-ppd(d+1) >= -max_diff, ''
#    m += ppd(d)-ppd(d+1) <= max_diff, ''


In [ ]:
#m.max_gap = 0.05
status = m.optimize(max_seconds=300)

if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))


In [ ]:
status

In [ ]:
m.write('model.lp')

In [ ]:
obj = m.objective_value
print(obj)

In [ ]:
fam_day_dict = {}

for i,v in enumerate(m.vars):
    
    if i%10000==0:
        print('{}/{}'.format(i,len(m.vars)))
    if abs(v.x) > 1e-6: # only printing non-zeros
        #print('{} : {}'.format(v.name, v.x))
        s = v.name.split(',')
        fam_day_dict[int(s[0][4:])] = int(s[1][4:])


In [ ]:
sel_series = pd.Series(fam_day_dict,name='assigned_day')

In [ ]:
len(sel_series)

In [ ]:
sel_series.hist(bins=[b for b in range(1,num_days+2,1)])

In [ ]:
#df2 = df2.join(sel_series)
df2['assigned_day'] = sel_series.astype(int)
#df2['assigned_day'] = df2.assigned_day.astype(int)
df2.head()

In [ ]:
df2[df2.assigned_day.isnull()]

In [ ]:
total_people = {}
for d in day:
    mask = df2['assigned_day']==d
    total_people[d] = df2[mask].n_people.sum()
    print(total_people[d])

Calculating the actual objective for the problem

In [ ]:
def accounting_penalty_actual(Nd,diff):
    return 300/max_people*(Nd-min_people)/400 * Nd**(0.5+np.fabs(diff)/50)

In [ ]:
total_accounting_penalty = sum([accounting_penalty_actual(total_people[d],total_people[d]-total_people[d+1])
                                if d<100 
                                else accounting_penalty_actual(total_people[d],0)
                                for d in day])
print(total_accounting_penalty)

In [ ]:
# Adding a column to the dataframe for the choice made...

def choice_func(r):
    if r['assigned_day'] in r.values[:10]:
        return choice[list(r.values[:10]).index(r.assigned_day)]
    else:
        return 'choice_10'

In [ ]:
df2['assigned_choice'] = df2.apply(choice_func,axis=1)
df2[df2.assigned_choice=='choice_4'].head()

In [ ]:
def simple_cost(r):
    return choice_gc[r['assigned_choice']] + r['n_people']*choice_pm[r['assigned_choice']]

In [ ]:
total_simple_cost = df2.apply(simple_cost,axis=1).sum()
total_simple_cost

In [ ]:
final_score = total_simple_cost + total_accounting_penalty
print('Final Score: {}'.format(final_score))

In [ ]:
df2['assigned_day'].to_csv('submission_80180.csv',header=True)

with a limit on the diff of 30 and solving for 500 seconds, the final score was 83115.97

In [ ]:
plt.plot(list(total_people.keys()),list(total_people.values()))

In [ ]:
# Creating penalties related to the days that have the highest consumption with no accounting

# sort the days by the number of visitors
sorted_days = {k: v for k, v in sorted(total_people.items(), key=lambda item: item[1],reverse=True)}

plt.plot([i for i in range(100)],list(sorted_days.values()))

In [ ]:
# Write the penalties to file
with open('day_penalty.txt','w') as fout:
    fout.write('day\tpenalty\n')
    for k,v in sorted_days.items():
        fout.write('{}\t{}\n'.format(k,v/125))
        print('{}\t{}'.format(k,v/125))
        
        
    